In [1]:
from keras.applications.vgg16 import VGG16

class YourClass:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build_model(self):
        vgg16 = VGG16(weights='imagenet', input_shape=self.input_shape, classes=self.num_classes, include_top=False)
        for layer in vgg16.layers:
            layer.trainable = False
        return vgg16


In [27]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5

# Load VGG16 model with pre-trained ImageNet weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Customize top layers
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Prepare data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './data/',
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        './data/',
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')



Found 22017 images belonging to 5 classes.
Found 22017 images belonging to 5 classes.


In [28]:
# Train the model
model.fit(
        train_generator,
        steps_per_epoch=2000 // 32, 
        epochs=10, 
        validation_data=validation_generator,
        validation_steps=800 // 32,  
        verbose=1) 




Epoch 1/10
62/62 [==============================] - 260s 4s/step - loss: 0.6907 - accuracy: 0.9582 - val_loss: 0.1001 - val_accuracy: 0.9737
Epoch 2/10
62/62 [==============================] - 256s 4s/step - loss: 0.1623 - accuracy: 0.9783 - val_loss: 0.0823 - val_accuracy: 0.9862
Epoch 3/10
62/62 [==============================] - 254s 4s/step - loss: 0.0654 - accuracy: 0.9879 - val_loss: 0.0693 - val_accuracy: 0.9825
Epoch 4/10
62/62 [==============================] - 255s 4s/step - loss: 0.0484 - accuracy: 0.9899 - val_loss: 0.0478 - val_accuracy: 0.9912
Epoch 5/10
62/62 [==============================] - 261s 4s/step - loss: 0.0367 - accuracy: 0.9924 - val_loss: 0.0054 - val_accuracy: 0.9987
Epoch 6/10
62/62 [==============================] - 253s 4s/step - loss: 0.0963 - accuracy: 0.9803 - val_loss: 0.2764 - val_accuracy: 0.9050
Epoch 7/10
62/62 [==============================] - 246s 4s/step - loss: 0.0546 - accuracy: 0.9854 - val_loss: 0.0074 - val_accuracy: 0.9987
Epoch 8/10
62

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

665/665 [==============================] - 1728s 3s/step - loss: 0.0183 - accuracy: 0.9956
Test Accuracy: 99.56%


In [29]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess a new image
img_path = r"./validate_img/test_1.webp "
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  

predictions = model.predict(img_array)

predicted_class_index = np.argmax(predictions[0])

class_labels =  ['docs','handwritten_docs', 'People', 'pets', 'signatures']
predicted_class = class_labels[predicted_class_index]
print(f'The predicted class is: {predicted_class}')

1/1 [==============================] - 0s 333ms/step
The predicted class is: pets


In [34]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = r"./validate_img/me.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0 

predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions[0])

class_labels = ['docs','handwritten_docs', 'People', 'pets', 'signatures']
predicted_class = class_labels[predicted_class_index]
print(f'The predicted class is: {predicted_class}')

1/1 [==============================] - 0s 135ms/step
The predicted class is: People


In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [36]:
model.save('vgg_model.h5')

In [32]:
#To DO list 

In [ ]:
"""
Make more data for docs and handwritten docs because the model is confusing b/w the docs and handwritten docs 
Model is performing poor when giving multiple faces image it is classifying as pets #Done

"""